In [2]:
from pymongo import MongoClient
from datetime import datetime as dt

from src.shared import Guild, Channel, ChannelType, Message, User, login_utils
from src.server import db, asset_generator

In [2]:
# dangerous!!!
db.guilds.drop()
db.channels.drop()
db.messages.drop()
db.users.drop()
db.passwords.drop()

In [3]:
db.messages.create_index("channel_id")
db.users.create_index("username", unique=True)
db.channels.create_index("guild_id")

'guild_id_1'

In [3]:
u = asset_generator.generate_user("Test_User", login_utils.hash_password("Password123"))

In [5]:
guilds = [asset_generator.generate_guild(f"Test guild {i}", u.id) for i in range(5)]

In [6]:
for guild in guilds:
    for i in range(5):
        asset_generator.generate_channel(f"test channel {i}", ChannelType.TEXT, guild.id)

In [7]:
for channel_db in db.channels.find():
    channel: Channel = Channel.from_db_dict(channel_db)
    buffer: list[dict] = []
    for i in range(150):
        id = asset_generator.get_id()
        buffer.insert(0, Message(id, channel.id, f"test message {i} in channel {channel.name} in guild {channel.guild_id}", attachment_id=0, author=u, timestamp=id >> asset_generator.TAG_LENGTH).to_db_dict())
    db.messages.insert_many(buffer)


In [8]:
db.get_messages(channel.id, 0, 10)

[Message(id=7190675527306948, channel_id=7190675527303198, content='test message 149 in channel test channel 4 in guild 7190675527303173', attachment_id=0, author=User(id=7190675527303168, username='Test_User'), timestamp=1714390642),
 Message(id=7190675527306947, channel_id=7190675527303198, content='test message 148 in channel test channel 4 in guild 7190675527303173', attachment_id=0, author=User(id=7190675527303168, username='Test_User'), timestamp=1714390642),
 Message(id=7190675527306946, channel_id=7190675527303198, content='test message 147 in channel test channel 4 in guild 7190675527303173', attachment_id=0, author=User(id=7190675527303168, username='Test_User'), timestamp=1714390642),
 Message(id=7190675527306945, channel_id=7190675527303198, content='test message 146 in channel test channel 4 in guild 7190675527303173', attachment_id=0, author=User(id=7190675527303168, username='Test_User'), timestamp=1714390642),
 Message(id=7190675527306944, channel_id=7190675527303198, c